In [1]:
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# You only need to authenticate once per session.
auth.authenticate_user()

In [2]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_gbq

# below imports are used for pretty pandas dataframes and plots
from IPython.display import display, HTML
%matplotlib inline
plt.style.use('ggplot')

# Set up environment variables
project_id = 'genuine-box-350018'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id
pandas_gbq.context.project = project_id


# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
    return pd.io.gbq.read_gbq(
        query,
        project_id=project_id,
        dialect="standard"
    )

if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project at the top of this cell.')

  # test it works
df = run_query("""
SELECT *
FROM `physionet-data.mimic_core.patients`
WHERE subject_id = 10012853
""")
assert df.shape[0] >= 1, 'unable to query MIMIC!'
display(df)


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10012853,F,91,2175,2014 - 2016,NaT


**ICD-9 CODES**

Gathering all ICD-9 codes for the cohort. 

In [6]:
icd_data = run_query(f'''
SELECT fco.subject_id, fco.hadm_id, fco.stay_id, icd.icd_code
FROM `physionet-data.mimic_hosp.diagnoses_icd` icd
INNER JOIN `genuine-box-350018.rnn_dataset.final-cohort-table` fco
ON fco.hadm_id = icd.hadm_id
''')
display(icd_data)

,subject_id,hadm_id,stay_id,icd_code
0,16816440,25634666,30641268,3079
1,16816440,25634666,30641268,4263
2,16816440,25634666,30641268,42611
3,16816440,25634666,30641268,E9444
4,16816440,25634666,30641268,36960
...,...,...,...,...
306089,19035431,29983439,31214569,V5861
306090,13915169,25293722,39785216,V5861
306091,10509045,20202063,39499821,V5861
306092,15992872,21168164,30351717,V5861


Creating binary columns for all 20 categories of ICD-9 Codes as listed in Table 2 of Supplementary Info.

In [7]:
icd_data = run_query(f'''
WITH icd_binary AS(
  SELECT fco.subject_id, fco.hadm_id, fco.stay_id, icd.icd_code
  , CASE WHEN substring(icd.icd_code,1,1) IN ("V", "E") THEN 0 ELSE 1 END AS icd_num
  FROM `physionet-data.mimic_hosp.diagnoses_icd` icd
  INNER JOIN `genuine-box-350018.rnn_dataset.final-cohort-table` fco
  ON fco.hadm_id = icd.hadm_id
)

SELECT icd.subject_id, icd.hadm_id, icd.stay_id, icd.icd_code
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 1 AND 139 THEN 1 ELSE 0 END) AS icd_1
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 140 AND 239 THEN 1 ELSE 0 END) AS icd_2
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 240 AND 279 THEN 1 ELSE 0 END) AS icd_3
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 280 AND 289 THEN 1 ELSE 0 END) AS icd_4
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 290 AND 319 THEN 1 ELSE 0 END) AS icd_5
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 320 AND 389 THEN 1 ELSE 0 END) AS icd_6
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 390 AND 459 THEN 1 ELSE 0 END) AS icd_7
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 460 AND 519 THEN 1 ELSE 0 END) AS icd_8
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 520 AND 579 THEN 1 ELSE 0 END) AS icd_9
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 580 AND 629 THEN 1 ELSE 0 END) AS icd_10
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 630 AND 677 THEN 1 ELSE 0 END) AS icd_11
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 680 AND 709 THEN 1 ELSE 0 END) AS icd_12
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 710 AND 739 THEN 1 ELSE 0 END) AS icd_13
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 740 AND 759 THEN 1 ELSE 0 END) AS icd_14
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 780 AND 789 THEN 1 ELSE 0 END) AS icd_15
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 790 AND 796 THEN 1 ELSE 0 END) AS icd_16
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 797 AND 799 THEN 1 ELSE 0 END) AS icd_17
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 800 AND 999 THEN 1 ELSE 0 END) AS icd_18
, (CASE WHEN icd.icd_num = 0 AND LOWER(substring(icd.icd_code,1,3)) LIKE 'v%' THEN 1 ELSE 0 END) AS icd_19
, (CASE WHEN icd.icd_num = 0 AND LOWER(substring(icd.icd_code,1,3)) LIKE 'e%' THEN 1 ELSE 0 END) AS icd_20
FROM `icd_binary` icd
ORDER BY subject_id
''')
display(icd_data)

,subject_id,hadm_id,stay_id,icd_code,icd_1,icd_2,icd_3,icd_4,icd_5,icd_6,...,icd_11,icd_12,icd_13,icd_14,icd_15,icd_16,icd_17,icd_18,icd_19,icd_20
0,10000980,26913865,39765666,41071,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000980,26913865,39765666,42823,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000980,26913865,39765666,41412,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000980,26913865,39765666,5854,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000980,26913865,39765666,4240,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306089,19999987,23865745,36195440,36846,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
306090,19999987,23865745,36195440,7810,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
306091,19999987,23865745,36195440,2449,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
306092,19999987,23865745,36195440,5849,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Aggregating by MAX occurence of each ICD Code (Max one billing) for each stay.

In [8]:
icd_data = run_query(f'''
WITH icd_group AS (

  WITH icd_binary AS(
  SELECT fco.subject_id, fco.hadm_id, fco.stay_id, icd.icd_code
  , CASE WHEN substring(icd.icd_code,1,1) IN ("V", "E") THEN 0 ELSE 1 END AS icd_num
  FROM `physionet-data.mimic_hosp.diagnoses_icd` icd
  INNER JOIN `genuine-box-350018.rnn_dataset.final-cohort-table` fco
  ON fco.hadm_id = icd.hadm_id
)

SELECT icd.subject_id, icd.hadm_id, icd.stay_id, icd.icd_code
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 1 AND 139 THEN 1 ELSE 0 END) AS icd_1
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 140 AND 239 THEN 1 ELSE 0 END) AS icd_2
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 240 AND 279 THEN 1 ELSE 0 END) AS icd_3
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 280 AND 289 THEN 1 ELSE 0 END) AS icd_4
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 290 AND 319 THEN 1 ELSE 0 END) AS icd_5
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 320 AND 389 THEN 1 ELSE 0 END) AS icd_6
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 390 AND 459 THEN 1 ELSE 0 END) AS icd_7
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 460 AND 519 THEN 1 ELSE 0 END) AS icd_8
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 520 AND 579 THEN 1 ELSE 0 END) AS icd_9
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 580 AND 629 THEN 1 ELSE 0 END) AS icd_10
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 630 AND 677 THEN 1 ELSE 0 END) AS icd_11
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 680 AND 709 THEN 1 ELSE 0 END) AS icd_12
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 710 AND 739 THEN 1 ELSE 0 END) AS icd_13
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 740 AND 759 THEN 1 ELSE 0 END) AS icd_14
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 780 AND 789 THEN 1 ELSE 0 END) AS icd_15
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 790 AND 796 THEN 1 ELSE 0 END) AS icd_16
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 797 AND 799 THEN 1 ELSE 0 END) AS icd_17
, (CASE WHEN icd.icd_num = 1 AND CAST(substring(icd.icd_code,1,3) AS int) BETWEEN 800 AND 999 THEN 1 ELSE 0 END) AS icd_18
, (CASE WHEN icd.icd_num = 0 AND LOWER(substring(icd.icd_code,1,3)) LIKE 'v%' THEN 1 ELSE 0 END) AS icd_19
, (CASE WHEN icd.icd_num = 0 AND LOWER(substring(icd.icd_code,1,3)) LIKE 'e%' THEN 1 ELSE 0 END) AS icd_20
FROM `icd_binary` icd

)

SELECT subject_id, hadm_id, stay_id
, MAX(icd_1) AS icd_1, MAX(icd_2) AS icd_2, MAX(icd_3) AS icd_3, MAX(icd_4) AS icd_4, MAX(icd_5) AS icd_5
, MAX(icd_6) AS icd_6, MAX(icd_7) AS icd_7, MAX(icd_8) AS icd_8, MAX(icd_9) AS icd_9, MAX(icd_10) AS icd_10
, MAX(icd_11) AS icd_11, MAX(icd_12) AS icd_12, MAX(icd_13) AS icd_13, MAX(icd_14) AS icd_14, MAX(icd_15) AS icd_15
, MAX(icd_16) AS icd_16, MAX(icd_17) AS icd_17, MAX(icd_18) AS icd_18, MAX(icd_19) AS icd_19, MAX(icd_20) AS icd_20
FROM `icd_group` 
GROUP BY subject_id, hadm_id, stay_id
ORDER BY subject_id
''')

display(icd_data)

,subject_id,hadm_id,stay_id,icd_1,icd_2,icd_3,icd_4,icd_5,icd_6,icd_7,...,icd_11,icd_12,icd_13,icd_14,icd_15,icd_16,icd_17,icd_18,icd_19,icd_20
0,10000980,26913865,39765666,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,10001217,24597018,37067082,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
2,10001725,25563031,31205490,0,0,0,0,1,1,1,...,0,0,1,0,1,0,0,1,1,1
3,10002155,23822395,33685454,0,1,1,1,1,0,1,...,0,0,0,1,1,0,1,1,0,1
4,10002223,22494570,39638202,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21887,19999297,21439025,37364566,1,0,1,1,1,0,0,...,0,0,0,0,1,1,0,0,0,0
21888,19999442,26785317,32336619,1,0,0,0,1,1,1,...,0,0,0,0,1,0,0,1,1,0
21889,19999625,25304202,31070865,0,0,1,1,1,1,1,...,0,1,0,0,1,0,0,0,1,0
21890,19999840,21033226,38978960,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,1


In [9]:
# Check if there are any patients with no billing codes assigned
sum(x==0 for x in icd_data.iloc[:,list(range(3,23))].sum(axis=1))

0

In [10]:
pandas_gbq.to_gbq(icd_data,'rnn_dataset.icd-table', project_id=project_id, if_exists="replace")

1it [00:08,  8.15s/it]


In [11]:
# Counting unique admissions to make sure there are no duplicates
unique_admissions = len(set(icd_data["hadm_id"]))
print(unique_admissions)

21892
